# Build An Agent
[build an agent](https://python.langchain.com/docs/tutorials/agents/)


**简介：**<br>
本节介绍了以下内容：<br>
- 介绍了为LLM添加工具（这里使用 `Tavily` ）的方法：使用 `model.bind_tools(tools)` 方法为LLM提供工具
- 介绍了利用 `create_react_agent` 快速构建一个简单的具有记忆功能的 `ReAct` 风格的 agent 的方法，并通过设置 `stream_mode` 来控制模型的输出方式（eg. 流式输出...）


> 这里简要说明一下，为什么还要再搭建一个 agent （个人理解）<br>
> 下面可以看到，当我问模型“某地天气是什么样”的时候，它告诉我它无法获取实时天气信息，尽管我已经通过 `model.bind_tools(tools)` 为它配置了工具<br>
> 当我输入“查询某地天气”时，它才调用工具。但返回的是结构化的，从工具获取的原生信息，缺乏总结等。当然，也可以通过在 `chains` 中增加流程实现.<br>
> 但这里有更简单的方法，agent是利用大模型的推理能力，决定是否要调用某个工具，调用完成后，用工具获取的外部知识回答用户输入问题



**使用Tavily作为工具**

> Tavily's Search API is a search engine built specifically for AI agents (LLMs), delivering real-time, accurate, and factual results at speed.<br>
> 如果没有API，请注册申请一个（[Tavily API Platform](https://app.tavily.com/home)）并配置好环境变量


## 1. 定义tools

In [2]:
import os
from langchain_tavily import TavilySearch

os.environ.get('TAVILY_API_KEY')
search = TavilySearch(max_result=2)
search_results = search.invoke("what's the wether in BeiJing")
print(search_results)

tools = [search]  # 先把所有工具放在一个列表里


{'query': "what's the wether in BeiJing", 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://weather.metoffice.gov.uk/forecast/wx4g092se', 'title': 'Beijing (China) weather - Met Office', 'content': '# Beijing (China) weather ## Find a forecast ## Forecast days Cloudy changing to sunny intervals by late morning. Sunny changing to partly cloudy in the afternoon. Cloudy changing to sunny intervals by late morning. Sunny changing to partly cloudy in the afternoon. ## 7 day forecast for Beijing | Feels like temperature (°C) | 24° | 23° | 23° | | Wind direction and speed (mph) | SW  1 | WNW  1 | NW  1 | NNW  2 | NNW  1 | NNW  1 | NNW  1 | NNW  1 | NW  1 | NNW  1 | N  1 | NE  2 | ENE  3 | E  3 | ESE  3 | SE  3 | SE  4 | SSE  4 | SSE  5 | SSE  5 | SSE  5 | S  5 | S  4 | S  3 | weather map ## Nearest forecasts * Beijing International 5.9 miles * WEATHER * WEATHER * WEATHER', 'score': 0.63125396, 'raw_content': None}, {'url': 'https://www.bbc.com/weather/181

## 2. 使用LLM

In [5]:
from langchain.chat_models import init_chat_model

os.environ['HTTP_PROXY'] = 'http://127.0.0.1:7890'
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7890'

os.environ.get('GOOGLE_API_KEY')

model = init_chat_model('gemini-2.5-flash', model_provider='google-genai')

query = 'hi!'
response = model.invoke([{'role': 'user', 'content': query}])
print(response.text())

Hi there! How can I help you today?


In [9]:
model_with_tools = model.bind_tools(tools)

query = 'tell me the wether in BeiJing.'
response = model_with_tools.invoke([{'role': 'user', 'content': query}])

print(f"message content: {response.text()}")
print(f'tools calss: {response.tool_calls}')

message content: I cannot tell you the weather in Beijing. I do not have access to real-time weather information.
tools calss: []


In [10]:
query = 'search for the wether in BeiJing.'
response = model_with_tools.invoke([{'role': 'user', 'content': query}])

print(f"message content: {response.text()}")
print(f'tools calss: {response.tool_calls}')

message content: 
tools calss: [{'name': 'tavily_search', 'args': {'query': 'weather in Beijing'}, 'id': 'b338a95e-0869-4ce0-9363-e39225a51fbb', 'type': 'tool_call'}]


## 3. create the agent

这里我们使用 `create_react_agent` 快速构建一个 ReAct 风格的代理<br>

如果想要实现更加复杂的业务流程，或者需要严格的流程，需要自己构建图，这些会在后面给予介绍

In [11]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

# 无状态查询（没有记忆）
input_message = {'role': 'user', 'content': 'hi!'}
response = agent_executor.invoke({'messages': [input_message]})

for message in response['messages']:
    message.pretty_print()

================================ Human Message =================================

hi!
================================== Ai Message ==================================

Hello! How can I help you today?


In [12]:
input_message = {'role': 'user', 'content': 'search for the wether in BeiJing.'}

response = agent_executor.invoke({'messages': [input_message]})
for message in response['messages']:
    message.pretty_print()

================================ Human Message =================================

search for the wether in BeiJing.
================================== Ai Message ==================================
Tool Calls:
  tavily_search (987f8b6f-e8cf-46d4-be78-035f4257b656)
 Call ID: 987f8b6f-e8cf-46d4-be78-035f4257b656
  Args:
    query: weather in Beijing
================================= Tool Message =================================
Name: tavily_search

{"query": "weather in Beijing", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in Beijing", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'Beijing', 'region': 'Beijing', 'country': 'China', 'lat': 39.9289, 'lon': 116.3883, 'tz_id': 'Asia/Shanghai', 'localtime_epoch': 1758895849, 'localtime': '2025-09-26 22:10'}, 'current': {'last_updated_epoch': 1758895200, 'last_updated': '2025-09-26 22:00', 'temp_c': 17.0, 'temp_f': 62.6, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': 

#### 1）streaming messages：按步骤输出

In [14]:
for step in agent_executor.stream({'messages': [input_message]}, stream_mode='values'):
    step['messages'][-1].pretty_print()

================================ Human Message =================================

search for the wether in BeiJing.
================================== Ai Message ==================================
Tool Calls:
  tavily_search (1b281606-868e-4b69-94de-4fcdb9d0528e)
 Call ID: 1b281606-868e-4b69-94de-4fcdb9d0528e
  Args:
    query: weather in Beijing
================================= Tool Message =================================
Name: tavily_search

{"query": "weather in Beijing", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in Beijing", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'Beijing', 'region': 'Beijing', 'country': 'China', 'lat': 39.9289, 'lon': 116.3883, 'tz_id': 'Asia/Shanghai', 'localtime_epoch': 1758895849, 'localtime': '2025-09-26 22:10'}, 'current': {'last_updated_epoch': 1758895200, 'last_updated': '2025-09-26 22:00', 'temp_c': 17.0, 'temp_f': 62.6, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': 

#### 2）streaming tokens：按token输出

> langchain的流式输出似乎只对python3.11及以上版本支持。其它版本似乎不行......

In [16]:
config = {'configurable': {'thread_id': 'agent001'}}

for step, metadata in agent_executor.stream(
    {'messages': [input_message]}, config, stream_mode='messages'
):
    if metadata['langgraph_node'] == 'agent' and (text := step.text()):
        print(text, end='|')

The weather in Beijing is clear with a temperature of 17.0°C (62.6°F).|

#### 3）增加记忆

这里，虽然它的回复是不知道我的名字，但实际上模型是有记忆的。原因见：[langchain-based model memory](https://stackoverflow.com/questions/79776520/langchain-based-model-memory)

In [50]:
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage

memory = MemorySaver()

agent_executor = create_react_agent(model, tools, checkpointer=memory)
config = {'configurable': {'thread_id': 'agent003'}}

for step in agent_executor.stream(
    {'messages': [HumanMessage('Hi! I am Bob!')]}, config, stream_mode='values'
):
    step['messages'][-1].pretty_print()

================================ Human Message =================================

Hi! I am Bob!
================================== Ai Message ==================================

Hi Bob! How can I help you today?


In [51]:
state = memory.get(config)
history = state["channel_values"]["messages"]

# Add new input
new_input = HumanMessage("What's my name?")
full_context = {"messages": history + [new_input]}

for step in agent_executor.stream(full_context, config, stream_mode='values'):
    step['messages'][-1].pretty_print()


================================ Human Message =================================

What's my name?
================================== Ai Message ==================================

I do not know your name. You can tell it to me if you wish!
